In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt

purchases = pd.read_csv('../DATA/purchases.csv.gzip', compression = 'gzip')

products = pd.read_csv('../DATA/products.csv')

purchases = purchases.merge(products, how = 'left', on = 'product_id')

columns_set = ['034aca0659036e1d051ae21659f23d24', '14d373dff524d487d4c82dfc89a53d41',
               '15e7af62dd1d892ab46833a49f2fc1e3', '1d2939ba1df78782f8f691ececfad913',
               '26ae5a7da8528b6fa6f24d7eea177b10', '2ede06c1ee0d06c64ceacd385449800d',
               '35642addd4de9bfa9ee61cd1ed5a6a12', '36bd2cad677db76f7b61c42bdd1720f8',
               '3ef03403a0c30d150764412389cd0ee1', '3f9010c11e60146fcb47d1d32888e899',
               '4202626fcbe0081eb956c55637bce6af', '428e08386efc442b9d2aa07e163e9d4a',
               '52f13dac0cf5614110ed50db5907a91e', '63852cb2ab73eacc9e6e0360da268472',
               '6b4ae3f25d97fcf1cc2ab650ffc0eb4c', '6f624f4be07a0301421e318d7acfafad',
               '703f4b6eb0bc1f9b1ca3c5042e7833b6', '70ac384d6a464b2280d3879891d592d6',
               '749c6194573eb35ac4c6a91726bca0e7', '79666b6e4451977f1c828265b1f16fe9',
               '825fe699abe9468dad9212716479b91b', '8a277809e5671841bb43f693cf5ea02a',
               '91fb12a54a21eb62e3744f064ebb8115', '93073a7304b280d23ecb306d341c94cd',
               'a2963967ff72fcb80047f2a09665e9cc', 'a49a930a4947960a174952aa2b06865f',
               'a75f1bad01688616b73e9ec3eba9450f', 'ad2b2e17d299351f9defb2169e5f9baf',
               'b0c4967fcee2c7db066508ff6d7f989a', 'be86806c6b3bed738e84fd3455451e25',
               'c2a3ea8d5ee314bce4de2da7477a3456', 'c3d6b407d9d028fe94177559537a711c',
               'c6e84cdac8f5ef64a97252c79497bfc0', 'd283080a934cb61d966ae29f486b5b8c',
               'de6f3b925a26e3cc5b5a5d97b43f84dd', 'e8705574ff46d35a4e941216c0542243',
               'ed2ad1797cd269992a5f5cbda9b97a70', 'f2333c90fbf58f51b3aece5f33b7b8d0',
               'f93982269d205571c4e94814aec8a29c', 'f980df59e1908f8f2d515e9af7cdb5fd',
               'fb3d79909eb455c6cd6183b956598a12', 'fb84f08028ecc803c1c36713282ec045']

%%time
pivot_table = pd.pivot_table(purchases, 
                             values='product_quantity', 
                             index=['num_transaction_id', 
                                    'transaction_datetime',
                                    'num_client_id'], 
                             columns = ['level_2'],
                             aggfunc = np.sum).fillna(0).reset_index()

pivot_table['transaction_datetime'] = pivot_table['transaction_datetime'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
pivot_table = pivot_table.set_index('transaction_datetime')

%%time
dataset = pivot_table[pivot_table.columns.drop('num_transaction_id')] \
    .groupby('num_client_id')[columns_set] \
    .rolling('30d') \
    .sum()

dataset.to_pickle('../DATA/dataset_30days.pickle')